# Portfolio Allocation using Deep Reinforcement Learning

## Part - I: Package Installations and Imports

Lets create the required directories first.

In [1]:
import os

folder_path = '/content/OptimizedCode'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")

else:
    print(f"Folder '{folder_path}' already exists.")

Folder '/content/OptimizedCode' created successfully.


Before executing the next cell, please upload the following files to the created `OptimizedCode` folder:

- install_packages.py
- data_scraping.py
- data_cleaning.py
- data_preprocessing.py

Also, upload `stock_tickers.xlsx` file to `/content/` path on Colab.

Now , lets install the necessary packages. The functtion below uses **multi-threading** for accelerated execution.

In [2]:
%%time
from OptimizedCode.install_packages import run_install_packages
run_install_packages()

CPU times: user 802 ms, sys: 134 ms, total: 936 ms
Wall time: 3min 28s


Now, we have too restart the runtime so that we can use all the packages.

In [ ]:
import os
os.kill(os.getpid(), 9)

Lets make the necessary imports now.

In [18]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import multiprocessing as mp

# from copy import deepcopy
from pyfolio import timeseries
# from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from OptimizedCode.data_scraping import run_scrape_data, scrape_data
from OptimizedCode.data_cleaning import run_clean_data, clean_data
from OptimizedCode.data_preprocessing import run_preprocess_data, process_data
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts

## Part - II: Data Scraping, Cleaning and Preprocessing

Lets scrape the data. We are using APIs from Yahoo Finance. It downloads all-time historical data of all the stocks listed in the S&P 500 Stock Index.

The function below uses **vectorization** for accelerated execution of Pandas Dataframe.

In [2]:
history = run_scrape_data()

[*********************100%%**********************]  503 of 503 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['COF']: OperationalError('database is locked')


Timer unit: 1e-09 s

Total time: 87.5962 s
File: /content/OptimizedCode/data_scraping.py
Function: scrape_data at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def scrape_data():
     9         1  421336295.0    4e+08      0.5      stock_tickers = pd.read_excel("/content/stock_tickers.xlsx")
    10         1     252414.0 252414.0      0.0      tickers = stock_tickers["Stock_Ticker"].to_list()
    11         1        9e+10    9e+10     98.0      hist = yf.download(tickers=tickers, period="max", interval='1d')
    12                                           
    13         1 1294468497.0    1e+09      1.5      dataset = pd.concat([hist.xs(key=ticker, level=1, axis=1).assign(Ticker=ticker) for ticker in tickers])
    14                                           
    15         1        703.0    703.0      0.0      return dataset



Now, lets clean this data. Here, I remove all the stock data whose history is less than 8000 entries as that would be too low for the RL model to work on. Also, I make sure that we only take the stock history of coinciding dates of the remaining stock tickers so as to maintain data availability for all the stock tickers on all dates.

The function below using **vectorization and multi-processing** to accelerate the execution.

In [3]:
cleaned_data = run_clean_data(history)

Timer unit: 1e-09 s

Total time: 10.0653 s
File: /content/OptimizedCode/data_cleaning.py
Function: clean_data at line 28

Line #      Hits         Time  Per Hit   % Time  Line Contents
    28                                           def clean_data(hist):
    29         1  638823618.0    6e+08      6.3      hist.dropna(inplace=True)
    30                                           
    31         1  295605226.0    3e+08      2.9      ticker_counts = hist.groupby('Ticker').size()
    32         1     688212.0 688212.0      0.0      valid_tickers = ticker_counts[ticker_counts >= 8000].index
    33         1  259925356.0    3e+08      2.6      hist = hist[hist['Ticker'].isin(valid_tickers)]
    34                                           
    35         1  143819788.0    1e+08      1.4      tickers = hist['Ticker'].unique()
    36         1      44937.0  44937.0      0.0      chunk_size = (len(tickers) // mp.cpu_count()) + 1
    37         1 6725239043.0    7e+09     66.8      common_dat

Now, lets pre-process the data. Here, I calculate the following technical indicators for all the stock tickers:

- MACD
- Bollinger Bands
- RSI
- ADL
- Ichimoku Cloud

I also calculate the covariance matrix for each stock ticker with each other for a history of one year (252 trading days). All these indicators would be used as 'States' for my RL environment.

The function below uses **vectorization and Numba** for accelerated execution.

In [4]:
preprocessed_data = run_preprocess_data(cleaned_data)

Timer unit: 1e-09 s

Total time: 32.0579 s
File: /content/OptimizedCode/data_preprocessing.py
Function: process_data at line 87

Line #      Hits         Time  Per Hit   % Time  Line Contents
    87                                           def process_data(cleaned_data):  
    88         1    4836753.0    5e+06      0.0      cleaned_data.reset_index(inplace=True) 
    89         1  420450667.0    4e+08      1.3      cleaned_data.sort_values(by=['Ticker', 'Date'], inplace=True)
    90         1     964198.0 964198.0      0.0      cleaned_data.set_index("Date", inplace=True)
    91                                           
    92         1     396091.0 396091.0      0.0      df_group = cleaned_data.groupby('Ticker')
    93                                           
    94         1 1490349906.0    1e+09      4.6      macd_data = df_group.apply(calculate_macd)
    95         1 1245656371.0    1e+09      3.9      bb_data = df_group.apply(calculate_bollinger_bands) 
    96         1 38455

Now, lets split the preprocesses data into train and test data. I am using a year's worth (252 trading days) of preprocessed data for testing. The rest goes for training the RL model.

In [5]:
test_index = preprocessed_data.index.unique()[-252:]
test_data = preprocessed_data.loc[test_index]

train_data = preprocessed_data.loc[~preprocessed_data.index.isin(test_index)]

train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)

test_data['Day'] = test_data.Day - test_data.Day.min()

test_data.index = test_data['Day']
train_data.index = train_data['Day']

## Part III: Designing Training Environment

Now, lets design the Stock Portfolio Environment. Our environment is Cython optimized and it also used multi-processsing for faster execution.

Lets load the cython module.

In [7]:
%load_ext Cython

Lets define the StockPortfolioEnv class now.

In [10]:
%%cython --annotate
import numpy as np
import pandas as pd
from gym import spaces, Env
import matplotlib.pyplot as plt
# from libc.math cimport sqrt
# from numba import njit
from copy import deepcopy
from numpy.random import default_rng
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
class StockPortfolioEnv(Env):
    """
    A single stock trading environment for OpenAI gym.
    """

    def __init__(self, object df, int stock_dim, int hmax, float initial_amount,
                 float transaction_cost_pct, float reward_scaling, int state_space,
                 int action_space, list tech_indicator_list, turbulence_threshold=None,
                 int lookback=252, int day=0):
        self.day = day
        self.lookback = lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct = transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        self.turbulence_threshold = turbulence_threshold if turbulence_threshold is not None else -1.0

        self.action_space = spaces.Box(low=0, high=1, shape=(action_space,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(state_space + len(tech_indicator_list), state_space), dtype=np.float32)

        while self.day not in self.df.index.unique():
            self.day += 1

        self.data = self.df.loc[self.day, :]
        self.covs = self.data['Cov_List'].values[0]
        self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
        self.terminal = False
        self.portfolio_value = self.initial_amount

        self.asset_memory = [self.initial_amount]
        self.portfolio_return_memory = [0]
        self.actions_memory = [[1 / stock_dim] * stock_dim]
        self.date_memory = [self.data.Date.unique()[0]]

    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.cdivision(True)
    def step(self, actions):
        cdef double mean_return, std_return, sharpe, portfolio_return, new_portfolio_value

        self.terminal = self.day >= self.df['Day'].max()

        if self.terminal:
            daily_returns = np.array(self.portfolio_return_memory, dtype=np.float64)
            df = pd.DataFrame(daily_returns, columns=['Daily_Return'])
            plt.plot(df['Daily_Return'].cumsum(), 'r')
            plt.savefig('cumulative_reward.png')
            plt.close()

            plt.plot(daily_returns, 'r')
            plt.savefig('rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            if daily_returns.std() != 0:
                mean_return = daily_returns.mean()
                std_return = daily_returns.std()
                sharpe = (252**0.5) * mean_return / std_return
                print("Sharpe: ", sharpe)
            print("=================================")

            return self.state, self.portfolio_value, self.terminal, {}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights.tolist())
            last_day_prices = self.data['Close'].values

            self.day += 1
            while self.day not in self.df.index.unique():
                self.day += 1

            self.data = self.df.loc[self.day,:]
            self.covs = self.data['Cov_List'].values[0]
            self.state = np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list], axis=0)
            current_prices = self.data['Close'].values

            returns = (current_prices / last_day_prices) - 1
            portfolio_return = (returns * weights).sum()

            new_portfolio_value = self.portfolio_value * (1 + portfolio_return)
            self.portfolio_value = new_portfolio_value

            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data['Date'].unique()[0])
            self.asset_memory.append(new_portfolio_value)

            self.reward = new_portfolio_value

            return self.state, self.reward, self.terminal, {}


    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.cdivision(True)
    def reset(self):

        cdef int day = 0
        cdef cnp.ndarray covs, state, action_memory
        cdef list tech_values
        cdef str tech

        self.asset_memory = [self.initial_amount]

        unique_days = self.df.index.unique()
        while day not in unique_days:
            day += 1

        self.day = day
        self.data = self.df.loc[self.day, :]

        covs = np.array(self.data['Cov_List'].values[0])
        tech_values = [self.data[tech].values.tolist() for tech in self.tech_indicator_list]
        state = np.append(covs, tech_values, axis=0)
        self.state = state
        self.portfolio_value = self.initial_amount

        self.terminal = False
        self.portfolio_return_memory = [0]

        action_memory = np.full((self.stock_dim,), 1.0 / self.stock_dim, dtype=np.float64)
        self.actions_memory = [action_memory.tolist()]
        self.date_memory = [self.data.Date.unique()[0]]

        return self.state


    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(numerator)
        softmax_output = numerator / denominator
        return softmax_output

    def save_asset_memory(self):
        return pd.DataFrame({'Date': self.date_memory, 'Daily_Return': self.portfolio_return_memory})

    def save_action_memory(self):
        df_actions = pd.DataFrame(self.actions_memory, index=pd.to_datetime(self.date_memory))
        df_actions.columns = self.data['Ticker'].values
        return df_actions

    def _seed(self, seed=None):

        self.np_random = default_rng(seed)
        return [seed]

    def get_sb_env(self):

        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

    def get_multiproc_env(self, n):

        def get_self():

            return deepcopy(self)

        e = SubprocVecEnv([get_self for _ in range(n)], start_method="fork")
        obs = e.reset()

        return e, obs

Content of stderr:
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_32560bd25b99003750a7e552f99569a3486fa815.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

Checking the state space for verification..

In [11]:
stock_dimension = len(train_data.Ticker.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 274, State Space: 274


Adding the technical indicators srom which the model the environment is going to fetch data to train the agent on.

In [12]:
tech_indicator_list = []

tech_indicator_list.append('MACD')
tech_indicator_list.append('Signal')
tech_indicator_list.append('MACD_Histogram')
tech_indicator_list.append('BB_Lower')
tech_indicator_list.append('BB_Upper')
tech_indicator_list.append('RSI')
tech_indicator_list.append('ADL')
tech_indicator_list.append('Tenkan Sen')
tech_indicator_list.append('Kijun Sen')
tech_indicator_list.append('Senkou Span A')
tech_indicator_list.append('Senkou Span B')

Now, lets create the environment instance for training.

In [13]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": tech_indicator_list,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train_data, **env_kwargs)

Converting that to a multi-processing instance..

In [14]:
n_cores = mp.cpu_count()
env_train, _ = e_train_gym.get_multiproc_env(n = n_cores)

Lets get the Actor-Critic agent instance for training our agent.

In [16]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cuda device


Now, lets finally train the model.

In [17]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

------------------------------------
| time/                 |          |
|    fps                | 145      |
|    iterations         | 100      |
|    time_elapsed       | 27       |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -389     |
|    explained_variance | 0        |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | 1.64e+09 |
|    reward             | 1.41e+06 |
|    std                | 0.999    |
|    value_loss         | 2.15e+13 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 149      |
|    iterations         | 200      |
|    time_elapsed       | 53       |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -389     |
|    explained_variance | 1.19e-07 |
|    learning_rate      | 0.0002   |
|    n_updates          | 199      |
|

It takes barely 5 mins to train on almost 2.1 million entries. Thats the magic of optimizations!

## Part - IV: Agent Evaluation

Now, lets test this on unseen data.

In [19]:
e_trade_gym = StockPortfolioEnv(df = test_data, **env_kwargs)
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:1000000.0
end_total_asset:1102035.1305933993
Sharpe:  0.8739654421514333
hit end!


We get a return of almost 11% and pretty high sharp ratio indicating safe investemnets.

Lets evaluate some performance metrics..

In [21]:
df_daily_return.rename(columns={'Date': 'date', 'Daily_Return': 'daily_return'}, inplace=True)
DRL_strat = convert_daily_return_to_pyfolio_ts(df_daily_return)
perf_func = timeseries.perf_stats
perf_stats_all = perf_func( returns=DRL_strat,
                              factor_returns=DRL_strat,
                                positions=None, transactions=None, turnover_denom="AGB")
print(perf_stats_all)

Annual return          0.102035
Cumulative returns     0.102035
Annual volatility      0.119565
Sharpe ratio           0.872230
Calmar ratio           0.815498
Stability              0.548846
Max drawdown          -0.125120
Omega ratio            1.150286
Sortino ratio          1.303938
Skew                   0.176992
Kurtosis               0.471442
Tail ratio             0.860400
Daily value at risk   -0.014650
Alpha                  0.000000
Beta                   1.000000
dtype: float64


These performance metrics provide a comprehensive evaluation of your RL-based portfolio allocator's performance. Let's break down the significance of each metric:

1. **Annual Return** and **Cumulative Returns**:
   - **Annual Return**: Represents the growth rate of the investment over a year. A return of 0.102035 (or 10.20%) indicates the percentage increase in the portfolio's value over a year.
   - **Cumulative Returns**: The total amount of return earned by an investment over a specific period. This value also indicates a 10.20% increase in the overall investment value.

2. **Annual Volatility**:
   - Measures the degree of variation of returns over time. A lower volatility (0.119565 in this case) suggests a more stable investment performance.

3. **Sharpe Ratio**:
   - The Sharpe ratio assesses the risk-adjusted return of an investment. A higher Sharpe ratio (0.872230) indicates better risk-adjusted returns.

4. **Calmar Ratio**:
   - The Calmar ratio evaluates the relationship between the average annual return and the maximum drawdown. It measures risk-adjusted returns relative to downside risk.

5. **Stability**:
   - This metric reflects the stability of the investment strategy. A value of 0.548846 suggests a moderate level of stability.

6. **Max Drawdown**:
   - Represents the maximum loss from a peak to a trough of a portfolio. A lower max drawdown (-0.125120) is preferable as it indicates less severe losses.

7. **Omega Ratio**:
   - Measures the ratio of expected gain to expected loss for an investment. A value greater than 1 (1.150286 here) is desirable.

8. **Sortino Ratio**:
   - Similar to the Sharpe ratio but considers only the downside risk (volatility of negative returns). A higher Sortino ratio (1.303938) indicates better risk-adjusted returns with a focus on downside volatility.

9. **Skew** and **Kurtosis**:
   - **Skew**: Measures the asymmetry of returns. A skew close to 0 (0.176992) indicates a relatively symmetrical distribution of returns.
   - **Kurtosis**: Measures the tails of the return distribution. A higher kurtosis (0.471442) suggests fatter tails in the return distribution.

10. **Tail Ratio**:
    - Compares the average return of the distribution to the downside deviation. A value less than 1 (0.860400) suggests higher downside risk relative to average return.

11. **Daily Value at Risk (VaR)**:
    - Estimates the maximum potential loss with a certain level of confidence (typically 95% or 99%). A VaR of -0.014650 means that there's a 95% confidence level that the worst daily loss will not exceed 1.465%.

12. **Alpha** and **Beta**:
    - **Alpha**: Measures the excess return of a portfolio relative to its benchmark after adjusting for market risk (Beta).
    - **Beta**: Indicates the sensitivity of the portfolio's returns to movements in the benchmark index. Beta of 1.000000 means the portfolio's returns move in line with the benchmark.